In [64]:
# Import Libraries and Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
import nltk
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
eng_stopwords = nltk.corpus.stopwords.words('english')

## Clean Articles Final Notebook 2

This notebook performs the following tasks:
    1. Standardizes timestamps
    2. Restrict date range to time-window of interest
    3. Preprocesses text for LSA/LSI
    4. Merges cleaned datasets for final processing    

In [65]:
articles = pd.read_csv("text_edit.csv").drop('Unnamed: 0', axis = 1)
articles = articles.sort_values('timeStamp', ascending = True)
articles.head()

,author,contents,description,publisher,source_url,timeStamp,title
0,Stripe.com,A complete payments platform engineered for gr...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support
1,Editorial Team,As it scrambles to serve a massively expanding...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...
2,Scott Scanlon,So many cryptocurrencies. So much money to be ...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...
3,http://www.dailymail.co.uk/home/search.html?s=...,By\n\nPress Association\n\t\nPublished:\n 19:...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...
4,Phil Glazer,Initial coin offerings (ICOs) are a regulatory...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...


__Standardize timestamps__

In [66]:
temp = pd.DatetimeIndex(articles['timeStamp']) # Gather all datetime objects
articles['date'] = temp.date                   # Pull out the date from the datetime objects and assign to Date column
articles['time'] = temp.time                   # Pull out the time from the datetime objects and assign to Time column
print(len(articles))
articles.tail(3)

20572


,author,contents,description,publisher,source_url,timeStamp,title,date,time
20569,Avivah Litan,\tA member of the Gartner Blog Network\tby Avi...,Blockchain sessions were in full force at the ...,Gartner.com,https://blogs.gartner.com/avivah-litan/2018/04...,2018-04-23 23:22:25+00:00,"Blockchain, Crypto, and Women; Musings from RSAC",2018-04-23,23:22:25
20570,Cyberparse.co.uk,Enlarge / Simplified figurative process of a C...,Enlarge / Simplified figurative process of a C...,Cyberparse.co.uk,http://cyberparse.co.uk/2018/04/23/new-hacks-s...,2018-04-23 23:32:00+00:00,New hacks siphon private cryptocurrency keys f...,2018-04-23,23:32:00
20571,pjbyrne,"Disclaimer: English lawyer, not practising thi...","Disclaimer: English lawyer, not practising thi...",Prestonbyrne.com,https://prestonbyrne.com/2018/04/23/on-ethereu...,2018-04-23 23:59:35+00:00,Whether Ethereum is a security,2018-04-23,23:59:35


__Restrict data set to date range under consideration__

In [67]:
# iterate over dates and drop those above range of interest
stop = datetime.date(year=2018,month=3,day=26)

for day in range(len(articles['date'])):
    check = articles['date'].loc[day]
    if check > stop:
        articles.drop(day, inplace = True)

In [68]:
print(len(articles))
articles.tail(3)

15888


,author,contents,description,publisher,source_url,timeStamp,title,date,time
15885,newsfeeds@nzherald.co.nz,\t NEW YORK (AP) — Twitter says it will ban or...,NEW YORK (AP) — Twitter says it will ban or re...,Nzherald.co.nz,http://www.nzherald.co.nz/business/news/articl...,2018-03-26 23:25:06+00:00,"Twitter to ban cryptocurrency ads, joining Fac...",2018-03-26,23:25:06
15886,Reuters,Sign up now to get free exclusive access to re...,Twitter Inc will start banning cryptocurrency ...,Cio.com.au,https://www.cio.com.au/article/635378/twitter-...,2018-03-26 23:43:50+00:00,Twitter to ban cryptocurrency ads,2018-03-26,23:43:50
15887,Reuters,Sign up now to get free exclusive access to re...,Twitter Inc will start banning cryptocurrency ...,Cio.com.au,https://www.cio.com.au/article/635378/twitter-...,2018-03-26 23:43:50+00:00,Twitter to ban cryptocurrency ads,2018-03-26,23:43:50


__Preprocess text for NLP formulations__

In [70]:
#Clean the articles - Remove stopwords, remove punctuation, all lowercase
cleaned_texts = []
for text in articles['contents']:
    text = re.sub('[^a-zA-Z]' ,'\n',text)
    text = [word for word in text.split() if not word in eng_stopwords]
    text = (' '.join(text))
    text = text.lower()
    cleaned_texts.append(text)
    
print(cleaned_texts[0])
print()
print(articles['contents'][0])

a complete payments platform engineered growth build scale recurring business model everything platforms need get sellers paid your business data fingertips the best way start internet business fight fraud machine learning share post twitter tom karlo january at stripe long excited possibilities cryptocurrencies experimentation innovation come in became first major payments company support bitcoin payments our hope bitcoin could become universal decentralized substrate online transactions help customers enable buyers places less credit card penetration use cases credit card fees prohibitive over past year two block size limits reached bitcoin evolved become better suited asset means exchange given overall success bitcoin community achieved hard quibble decisions made along way and certainly happy see novel ambitious project well this led bitcoin becoming less useful payments however transaction confirmation times risen substantially turn led increase failure rate transactions denominat

In [71]:
articles['contents'] = cleaned_texts
articles.head()

,author,contents,description,publisher,source_url,timeStamp,title,date,time
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02


__Combine cleaned articles with "Markers" from Time Series event detection__ 

In [74]:
markers = pd.read_csv("daily_bitcoin_price.csv")
markers

,date,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,2018-01-23,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,2018-01-24,10971.984729,10980.105139,10963.683576,10972.448472,10.366352,113846.298097,10971.714961,13699.514545,355122.398070,1708.583434
2,2018-01-25,11259.428354,11265.764250,11252.426514,11259.235854,9.321376,105256.012808,11259.205322,13584.255872,364361.017716,1753.032764
3,2018-01-26,10992.639417,10999.507153,10985.075500,10992.569951,12.927589,141304.100192,10992.283702,13430.223722,365297.124620,1757.536610
4,2018-01-27,11177.254542,11182.093500,11172.314326,11177.383819,8.325551,93366.121547,11177.335040,13299.159649,370485.506037,1782.499222
5,2018-01-28,11425.907035,11430.281958,11422.407681,11426.112271,9.252465,105956.501345,11426.139665,13198.271418,375054.817538,1804.483332
6,2018-01-29,11137.583674,11140.958229,11133.652896,11137.279299,7.619312,84888.235432,11137.324422,13102.589609,379154.300212,1824.206977
7,2018-01-30,10621.101118,10627.480681,10613.562326,10620.313118,18.900901,194549.235567,10620.605888,13025.871604,387245.629262,1863.136402
8,2018-01-31,9978.939889,9986.435667,9970.953299,9978.950660,13.351993,132254.567681,9978.619754,12914.256311,400836.959836,1928.527722
9,2018-02-01,9497.854597,9507.023528,9488.127375,9497.577813,26.646358,243622.934131,9497.523230,12793.302949,415649.903564,1999.796531


In [75]:
temp = pd.DatetimeIndex(markers['date']) #Gather all datetime objects
markers['date'] = temp.date #Pull out the date from the datetime objects and assign to Date column
markers.head()

,date,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,2018-01-23,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,2018-01-24,10971.984729,10980.105139,10963.683576,10972.448472,10.366352,113846.298097,10971.714961,13699.514545,355122.398070,1708.583434
2,2018-01-25,11259.428354,11265.764250,11252.426514,11259.235854,9.321376,105256.012808,11259.205322,13584.255872,364361.017716,1753.032764
3,2018-01-26,10992.639417,10999.507153,10985.075500,10992.569951,12.927589,141304.100192,10992.283702,13430.223722,365297.124620,1757.536610
4,2018-01-27,11177.254542,11182.093500,11172.314326,11177.383819,8.325551,93366.121547,11177.335040,13299.159649,370485.506037,1782.499222


In [76]:
# comibine dataframes 
df = articles.merge(markers, how='outer', on="date")
df

,author,contents,description,publisher,source_url,timeStamp,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,2018-01-23 00:00:00+00:00,Ending Bitcoin Support,2018-01-23,00:00:00,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,2018-01-23 00:01:00+00:00,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,2018-01-23 00:03:12+00:00,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,2018-01-23 00:05:47+00:00,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,2018-01-23 00:06:02+00:00,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
5,Financial Times,hannah murphy london bitflyer tokyo based oper...,Exchange to offer trading of bitcoin/euro pair...,Financial Times,https://www.ft.com/content/5bf1462c-ff84-11e7-...,2018-01-23 00:06:21+00:00,Japan’s bitFlyer set to stage Europe expansion,2018-01-23,00:06:21,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
6,Topix.com,not topix user yet forgot your password news p...,The fundamentals are as untenable as they were...,Topix.com,http://www.topix.com/tech/p2p/2018/01/dont-try...,2018-01-23 00:11:59+00:00,Don't Try To Catch The Bitcoin Knife,2018-01-23,00:11:59,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
7,Chloe Aiello,var postloadfunctions var foresee enabled var ...,ETF Managers Group Founder and CEO Sam Masucci...,CNBC,https://www.cnbc.com/2018/01/22/marijuana-etf-...,2018-01-23 00:26:00+00:00,Here's why one investor is way more comfortabl...,2018-01-23,00:26:00,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
8,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,Miami has apparently witnessed the “hottest” w...,Bitcoinist.com,http://bitcoinist.com/achain-showcased-collabo...,2018-01-23 00:30:10+00:00,Achain Showcased its Collaborative Platform at...,2018-01-23,00:30:10,10694.957153,10707.347083,10682.436090,10694.852736,15.001166,160812.535684,10694.836727,13808.057214,337446.018425,1623.537913
9,Sasha Lekach,the bitcoin craze officially jumped real estat...,The bitcoin craze has officially jumped to rea...,Mashable,http://mashable.com/2018/01/22/cryptohomes-rea...,2018-01-23 00:36:22+00:00,Homeowners everywhere are li

In [35]:
# fill missing values with int 0
#df['label'] = df['label'].fillna(0)
#df.index = np.arange(0,len(df))
#df.tail()

In [77]:
df.to_csv("article_data_and_price.csv")

___